<a href="https://colab.research.google.com/github/Sagaust/DH-Computational-Methodologies/blob/main/Loading_from_Colab_to_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install --upgrade google-cloud-bigquery
!pip install pandas

# Import libraries
from google.cloud import bigquery
import pandas as pd

# Authenticate to BigQuery
from google.colab import auth
auth.authenticate_user()

# Initialize BigQuery client
project_id = 'philedu-416212'
client = bigquery.Client(project=project_id)

# Define your dataset and table names
dataset_id = 'philCurriculum'
table_name = 'your_table_name'

# Read CSV files into pandas DataFrame
# Assuming you have CSV files named file1.csv, file2.csv, etc.
file_paths = ['file1.csv', 'file2.csv']  # Add more file paths if needed
dataframes = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate DataFrames if needed
concatenated_df = pd.concat(dataframes)

# Define BigQuery schema (you need to define it according to your CSV file structure)
schema = [
    bigquery.SchemaField(name, 'STRING') for name in concatenated_df.columns
]

# Upload DataFrame to BigQuery table
table_ref = client.dataset(dataset_id).table(table_name)
job_config = bigquery.LoadJobConfig(schema=schema, skip_leading_rows=1)
job = client.load_table_from_dataframe(concatenated_df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f'CSV files uploaded to BigQuery table {dataset_id}.{table_name} successfully.')


In [6]:
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [10]:
# Import the os module
import os
import pandas as pd

# Get list of CSV files in the default directory
file_paths = [f for f in os.listdir() if f.endswith('.csv') and os.path.isfile(f)]

# Iterate through CSV files and attempt to load each one
for file_path in file_paths:
    try:
        pd.read_csv(file_path, nrows=1)  # Attempt to read the first row to infer schema
    except Exception as e:
        print(f'Error reading file: {file_path}')
        print(f'Error message: {str(e)}')


In [16]:
from google.api_core.exceptions import NotFound

# Define the table name and schema
table_name = 'your_table_name'
schema = [
    bigquery.SchemaField('column1', 'STRING'),
    bigquery.SchemaField('column2', 'INTEGER'),
    # Add more fields as needed based on your data
]

# Create table reference
table_ref = dataset_ref.table(table_name)

# Check if table exists and delete it if it does
try:
    client.get_table(table_ref)
    client.delete_table(table_ref)
    print(f'Table {table_name} already exists and has been deleted.')
except NotFound:
    pass

# Create the table
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)

# Load data into the table
job_config = bigquery.LoadJobConfig(schema=schema, skip_leading_rows=1, max_bad_records=10)  # Adjust as needed
job = client.load_table_from_dataframe(concatenated_df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f'Table {table_name} created and data loaded successfully.')


ValueError: bq_schema contains fields not present in dataframe: {'column2', 'column1'}

In [17]:
!pip install google-cloud-bigquery

In [18]:
from google.cloud import bigquery
from google.colab import auth
import os

# Authenticate and create a client
auth.authenticate_user()
project_id = 'philedu-416212'  # Replace with your project ID
client = bigquery.Client(project=project_id)

# Define your BigQuery Dataset
dataset_id = 'philCurriculum'  # Replace with your dataset ID
dataset_ref = client.dataset(dataset_id)

# Directory where your CSV files are stored
directory = "/content/"  # Assuming all CSVs are in the '/content/' directory

# Iterate over each file in the directory and load it into BigQuery
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        table_id = filename.split('.')[0]  # Assuming table name is same as filename
        table_ref = dataset_ref.table(table_id)

        job_config = bigquery.LoadJobConfig()
        job_config.source_format = bigquery.SourceFormat.CSV
        job_config.skip_leading_rows = 1  # Set to 0 if no header row
        job_config.autodetect = True
        job_config.max_bad_records = 10  # Allows up to 10 bad records before failing


        with open(os.path.join(directory, filename), "rb") as source_file:
            job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

        job.result()  # Waits for the job to complete

        print(f"Loaded {filename} into {dataset_id}.{table_id}")


BadRequest: 400 Provided Schema does not match Table philedu-416212:philCurriculum.C972EE73-8DDF-E520-B940-E354349AB9F0. Cannot add fields (field: Date_Added); reason: invalid, message: Provided Schema does not match Table philedu-416212:philCurriculum.C972EE73-8DDF-E520-B940-E354349AB9F0. Cannot add fields (field: Date_Added); reason: invalid, message: It looks like you are appending to an existing table with autodetect enabled. Disabling autodetect may resolve this.